# Convolutional Neural Networks

This notebook introduces convolutional neural networks (CNNs), a more powerful classification model similar to the Neural Bag-of-Words (BOW) model you explored earlier.

## Outline

- **Part (a):** Model Architecture
- **Part (b):** Implementing the CNN Model
- **Part (c):** Tuning

In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst

# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


## (a) Model Architecture

CNNs are a more sophisticated neural model for sentence classification than the Neural BOW model we saw in the last section. CNNs operate by sweeping a collection of filters over a text. Each filter produces a sequence of feature values known as a _feature map_. In one of the most basic formulations introduced by [Kim (2014)](https://www.aclweb.org/anthology/D14-1181), a single layer of _pooling_ is used to summarize the _feature maps_ as a fixed length vector. The fixed length vector is then feed to the output layer in order to produce classification labels. A popular choice for the pooling operation is to take the maximum feature value from by each _feature map_.

![Convolutional Neural Network from Kim 2014](kim_2014_figure_1_cnn.png)
*CNN model architure, Figure 1 from Kim (2014)*

We'll use the following notation:
- $h$: filter/kernel length (in words)
- $w^{(i)} \in \mathbb{Z}$, the word id for the $i^{th}$ word of the sequence (as an integer index)
- $x^{(i)} \in \mathbb{R}^d$ for the vector representation (embedding) of $w^{(i)}$
- $x^{i:i+j}$ is the concatenation of $x^{(i)}, x^{(i+1)} ... x^{(i+j)}$ 
- $c^{(i)}_{k}$ is the value of the $k^{th}$ feature map at the $i^{th}$ position along the word sequence, each filter applies over a window of $h$ words and uses non-linearity $f$.
- $c_k$ is one feature map (the $k_{th}$).  Its values are $c^{(0)}_{k}$, $c^{(1)}_{k}$, $c^{(2)}_{k}$, ...
- $\hat{c}_{k}$ is the value of the $k^{th}$ feature after pooling the feature map over the whole sequence.
- $\hat{C}$ is the concatenation of pooled feature maps. 
- $y$ for the target label ($\in 1,\ldots,\mathtt{num\_classes}$)

Our model is defined as:
- **Embedding layer:** $x^{(i)} = W_{embed}[w^{(i)}]$
- **Convolutional layer:** $c^{(i)}_{k} = f(x^{i:i+h-1} W_k + b)$
- **Pooling layer:**  $\hat{c}_{k}$ = $max(c_k)$ = $max(c^{(0)}_{k}, c^{(1)}_{k}...)$ 
- **Output layer:** $\hat{y} = \hat{P}(y) = \mathrm{softmax}(\hat{C} W_{out} + b_{out})$


We'll refer to the first part of this model (**Embedding layer**, **Convolutional layer**, and **Pooling layer**) as the **Encoder**: it has the role of encoding the input sequence into a fixed-length vector representation that we pass to the output layer.

We'll also use these as shorthand for important dimensions:
- `V`: the vocabulary size (equal to `ds.vocab.size`)
- `N`: the maximum number of tokens in the input text
- `embed_dim`: the embedding dimension $d$
- `filters`: number filters per filter length
- `num_classes`: the number of target classes (2 for the binary task)

## (a) Short Answer Questions

When answering these questions in the answers file,
`embed_dim = 10`, `kernel_size = 3`, `filters=128`, `N=10` and `num_classes = 7`.  Assume a single example (no batching).

1. In terms of these values, the vocabulary size `V` and the maximum sequence length `N`, what are the
   shapes of the following variables: 
   $c^{(i)}_{k}$, $c_{k}$, $\hat{c}_{k}$, and $\hat{C}$. Assume a stride size of 1. Assume padding is not used (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='VALID'), provide the shapes listed above.
<p>
2. What are the shapes of $c_{k}$ and $\hat{c}_{k}$ when padding is used.
      (e.g., for tf.nn.max_pool and tf.nn.conv1d, setting padding='same').
<p>
3. How many parameters are in each of the convolutional filters, $W_{k}$?  What if the kernel size is 4? 5? And the output layer, $W_{out}$?
<p>
<p>
4. Historically NLP models made heavy use of manual feature engineering. In relation to systems with manually engineered features, describe what type of operation is being performed by the convolutional filters.
<p>
5. Suppose that we have two examples, `[foo bar baz]` and `[baz bar foo]`.  Does this model definitely make the same predictions on these? Why or why not?

## (b) Implementing the CNN Model

We'll implement our CNN model below. Our implementation will differ from [Kim (2014)](https://www.aclweb.org/anthology/D14-1181) in that we will support using multiple dense hidden layers after the convolutional layers.

**Before you start**, be sure to answer the short-answer questions above!

In [2]:
import sst

# Load SST dataset
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")
max_len = 40
train_x, train_ns, train_y = ds.as_padded_array('train', max_len=max_len, root_only=True)
dev_x,   dev_ns,   dev_y   = ds.as_padded_array('dev',   max_len=max_len, root_only=True)
test_x,  test_ns,  test_y  = ds.as_padded_array('test',  max_len=max_len, root_only=True)


Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


In [27]:
# Specify model hyperparameters.
epochs = 10
embed_dim = 5
num_filters = [2, 2, 2]
kernel_sizes = [2, 3, 4]
dense_layer_dims = []
dropout_rate = 0.8
num_classes = len(ds.target_names)

# Construct the convolutional neural network.
# The form of each keras layer function is as follows:
#    result = keras.layers.LayerType(arguments for the layer)(layer(s) it should use as input)
# concretely,
#    this_layer_output = keras.layers.Dense(100, activation='relu')(prev_layer_vector)
# performs this_layer_output = relu(prev_layer_vector x W + b) where W has 100 columns.

# Input is a special "layer".  It defines a placeholder that will be overwritten by the training data.
# In our case, we are accepting a list of wordids (padded out to max_len).
wordids = keras.layers.Input(shape=(max_len,))

# Embed the wordids.
# Recall, this is just a mathematically equivalent operation to a linear layer and a one-hot
h = keras.layers.Embedding(ds.vocab.size, embed_dim, input_length=max_len)(wordids)

# Construct "filters" randomly initialized filters with dimension "kernel_size" for each size of filter we want.
# With the default hyperparameters, we construct 10 filters each of size 2, 3, 4.  As in the image above, each filter
# is wide enough to span the whole word embedding (this is why the convolution is "1d" as seen in the
# function name below).
conv_layers_for_all_kernel_sizes = []
for kernel_size, filters in zip(kernel_sizes, num_filters):
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)

# Concat the feature maps from each different size.
h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

# Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values
# in the vector.
# See https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf for details.
h = keras.layers.Dropout(rate=dropout_rate)(h)

### YOUR CODE HERE
# Add a fully connected layer for each dense layer dimension in dense_layer_dims.
h = keras.layers.Dense(10, activation='relu')(h)
### END YOUR CODE

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=wordids, outputs=prediction)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

In [28]:
model.reset_states()
model.fit(train_x, train_y, epochs=epochs)

Train on 6920 samples
Epoch 1/10
6920/6920 [==============================] - 2s 273us/sample - loss: 0.6929 - acc: 0.5165
Epoch 2/10
6920/6920 [==============================] - 1s 168us/sample - loss: 0.6920 - acc: 0.5233
Epoch 3/10
6920/6920 [==============================] - 1s 165us/sample - loss: 0.6899 - acc: 0.5263
Epoch 4/10
6920/6920 [==============================] - 1s 180us/sample - loss: 0.6807 - acc: 0.5293
Epoch 5/10
6920/6920 [==============================] - 1s 197us/sample - loss: 0.6593 - acc: 0.5763
Epoch 6/10
6920/6920 [==============================] - 1s 200us/sample - loss: 0.6299 - acc: 0.6312
Epoch 7/10
6920/6920 [==============================] - 1s 202us/sample - loss: 0.6058 - acc: 0.6579
Epoch 8/10
6920/6920 [==============================] - 1s 215us/sample - loss: 0.5893 - acc: 0.6782
Epoch 9/10
6920/6920 [==============================] - 1s 198us/sample - loss: 0.5722 - acc: 0.6932
Epoch 10/10
6920/6920 [==============================] - 1s 210us/sam

## Evaluation

Call [evaluate](https://keras.io/models/model/#evaluate) on your model.

In [29]:
#### YOUR CODE HERE ####
model.evaluate(dev_x, dev_y)
#### END(YOUR CODE) ####

872/872 [==============================] - 1s 793us/sample - loss: 0.5894 - acc: 0.7053


[0.5893976754004803, 0.70527524]

# Part (c): Tuning Your Model

We'll once again want to optimize hyperparameters for our model to see if we can improve performance. The CNN model includes a number of new parameters that can significantly influence model performance.

In this section, you will be asked to describe the new parameters as well as use them to attempt to improve the performance of your model.

## Part (c) Short Answer Questions

  1. Choose two parameters unique the CNN model, perform at least 10 runs with different combinations of values for these parameters, and then report the dev set results below. ***Hint: Consider wrapping the training code above in a for loop the examines the different values.***  To do this efficiently, you should consider [this paper](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) from Bergstra and Bengio.  [This blog post](https://blog.floydhub.com/guide-to-hyperparameters-search-for-deep-learning-models/) also has a less formal treatment of the same topic.
  2. Describe any trends you see in experiments above (e.g., can you identify good ranges for the individual parameters; are there any interesting interactions?)
  3. Pick the three best configurations according to the dev set and evaluate them on the test data. Is the ranking of the three best models the same on the dev and test sets?
  4. What was the best accuracy you achieved on the test set?

In [30]:
def build_cnn_model(num_filters=[2, 2, 2], kernel_sizes=[2, 3, 4]):
    
    embed_dim=5
    dropout_rate = 0.8
    num_classes = 2

    # Construct the convolutional neural network.
    # The form of each keras layer function is as follows:
    #    result = keras.layers.LayerType(arguments for the layer)(layer(s) it should use as input)
    # concretely,
    #    this_layer_output = keras.layers.Dense(100, activation='relu')(prev_layer_vector)
    # performs this_layer_output = relu(prev_layer_vector x W + b) where W has 100 columns.

    # Input is a special "layer".  It defines a placeholder that will be overwritten by the training data.
    # In our case, we are accepting a list of wordids (padded out to max_len).
#     print(num_filters)
#     print(kernel_sizes)
    wordids = keras.layers.Input(shape=(max_len,))

    # Embed the wordids.
    # Recall, this is just a mathematically equivalent operation to a linear layer and a one-hot
    h = keras.layers.Embedding(ds.vocab.size, embed_dim, input_length=max_len)(wordids)

    # Construct "filters" randomly initialized filters with dimension "kernel_size" for each size of filter we want.
    # With the default hyperparameters, we construct 10 filters each of size 2, 3, 4.  As in the image above, each filter
    # is wide enough to span the whole word embedding (this is why the convolution is "1d" as seen in the
    # function name below).
    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)

    # Concat the feature maps from each different size.
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)

    # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values
    # in the vector.
    # See https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf for details.
    h = keras.layers.Dropout(rate=dropout_rate)(h)

    ### YOUR CODE HERE
    # Add a fully connected layer for each dense layer dimension in dense_layer_dims.
    h = keras.layers.Dense(10, activation='relu')(h)
    ### END YOUR CODE

    prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

    model = keras.Model(inputs=wordids, outputs=prediction)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # From information theory notebooks.
                  metrics=['accuracy'])
    
    return model

In [33]:
from random import randint
from sklearn.model_selection import RandomizedSearchCV

epochs = 10

model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=build_cnn_model, epochs=epochs) 

# Specify parameters and distributions to sample from
num_filters = []
for i in range(1,7):
    num_filters.append([i,i,i])

kernel_sizes = []
for i in range(1,16):
    kernel_sizes.append([2, randint(2,5), randint(2,5)])

param_dict = dict(num_filters=num_filters, 
                  kernel_sizes=kernel_sizes)


n_iter_search = 5 # Number of parameter settings that are sampled.

grid_search = GridSearchCV(estimator=model, 
                             param_distributions=param_dict,
                             n_iter=n_iter_search)
grid_search.fit(train_x, train_y)

/home/erik_hou/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Train on 4613 samples
Epoch 1/10
4613/4613 [==============================] - 2s 426us/sample - loss: 0.6448 - acc: 0.6672
Epoch 2/10
4613/4613 [==============================] - 1s 213us/sample - loss: 0.6218 - acc: 0.6885
Epoch 3/10
4613/4613 [==============================] - 1s 221us/sample - loss: 0.6140 - acc: 0.6885
Epoch 4/10
4613/4613 [==============================] - 1s 227us/sample - loss: 0.6084 - acc: 0.6885
Epoch 5/10
4613/4613 [==============================] - 1s 257us/sample - loss: 0.5962 - acc: 0.6885
Epoch 6/10
4613/4613 [==============================] - 1s 227us/sample - loss: 0.5753 - acc: 0.6887
Epoch 7/10
4613/4613 [==============================] - 1s 215us/sample - loss: 0.5372 - acc: 0.6917
Epoch 8/10
4613/4613 [==============================] - 1s 202us/sample - loss: 0.5035 - acc: 0.7117
Epoch 9/10
4613/4613 [==============================] - 1s 235us/sample - loss: 0.4647 - acc: 0.7537
Epoch 10/10
2307/2307 [==============================] - 1s 521us/sam

4613/4613 [==============================] - 1s 236us/sample - loss: 0.6789 - acc: 0.5636
Epoch 5/10
4613/4613 [==============================] - 1s 220us/sample - loss: 0.6688 - acc: 0.5732
Epoch 6/10
4613/4613 [==============================] - 1s 218us/sample - loss: 0.6466 - acc: 0.6007
Epoch 7/10
4613/4613 [==============================] - 1s 225us/sample - loss: 0.6073 - acc: 0.6432
Epoch 8/10
4613/4613 [==============================] - 1s 236us/sample - loss: 0.5739 - acc: 0.6766
Epoch 9/10
4613/4613 [==============================] - 1s 241us/sample - loss: 0.5398 - acc: 0.6974
Epoch 10/10
2307/2307 [==============================] - 1s 537us/sample - loss: 0.5727 - acc: 0.7195
Train on 4614 samples
Epoch 1/10
4614/4614 [==============================] - 2s 480us/sample - loss: 0.6206 - acc: 0.7156
Epoch 2/10
4614/4614 [==============================] - 1s 243us/sample - loss: 0.5974 - acc: 0.7204
Epoch 3/10
4614/4614 [==============================] - 1s 225us/sample - loss:

4614/4614 [==============================] - 1s 223us/sample - loss: 0.5701 - acc: 0.7208
Epoch 8/10
4614/4614 [==============================] - 1s 235us/sample - loss: 0.5626 - acc: 0.7202
Epoch 9/10
4614/4614 [==============================] - 1s 233us/sample - loss: 0.5559 - acc: 0.7254
Epoch 10/10
2306/2306 [==============================] - 1s 626us/sample - loss: 1.0022 - acc: 0.1418
Train on 6920 samples
Epoch 1/10
6920/6920 [==============================] - 3s 443us/sample - loss: 0.6930 - acc: 0.5120
Epoch 2/10
6920/6920 [==============================] - 2s 255us/sample - loss: 0.6900 - acc: 0.5396
Epoch 3/10
6920/6920 [==============================] - 2s 257us/sample - loss: 0.6802 - acc: 0.5790
Epoch 4/10
6920/6920 [==============================] - 2s 239us/sample - loss: 0.6496 - acc: 0.6461
Epoch 5/10
6920/6920 [==============================] - 2s 264us/sample - loss: 0.6031 - acc: 0.6961
Epoch 6/10
6920/6920 [==============================] - 2s 267us/sample - loss:

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f75ef8eda10>,
                   iid='warn', n_iter=5, n_jobs=None,
                   param_distributions={'kernel_sizes': [[2, 2, 3], [2, 4, 4],
                                                         [2, 5, 5], [2, 5, 4],
                                                         [2, 4, 4], [2, 2, 2],
                                                         [2, 5, 4], [2, 2, 5],
                                                         [2, 4, 4], [2, 3, 5],
                                                         [2, 5, 4], [2, 2, 3],
                                                         [2, 4, 4], [2, 3, 3],
                                                         [2, 5, 3]],
                                        'num_filters': [[1, 1, 1], [2, 2, 2],
                                                        [3, 3, 3

In [34]:
# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.552168 using {'num_filters': [6, 6, 6], 'kernel_sizes': [2, 4, 4]}
0.552168 (0.098488) with: {'num_filters': [6, 6, 6], 'kernel_sizes': [2, 4, 4]}
0.286272 (0.229576) with: {'num_filters': [1, 1, 1], 'kernel_sizes': [2, 2, 3]}
0.455491 (0.240878) with: {'num_filters': [3, 3, 3], 'kernel_sizes': [2, 3, 3]}
0.523844 (0.182007) with: {'num_filters': [4, 4, 4], 'kernel_sizes': [2, 5, 5]}
0.340029 (0.232175) with: {'num_filters': [2, 2, 2], 'kernel_sizes': [2, 2, 3]}
